In [1]:
from flair.models import SequenceTagger
from flair.data import Sentence


tagger = SequenceTagger.load('ner')
def extract_person_names(text,company,url):
    try:
        sentence = Sentence(text)
        tagger.predict(sentence)
        person_names = set(entity.text for entity in sentence.get_spans('ner') if entity.tag == 'PER')
        print(f'Done for company {company} and webpage {url}')
        ans =  list(person_names)

        ans = [s.replace(company, "") for s in ans]
        return ans
    except Exception as e:
        print(e)

        return []



2024-08-24 19:24:59,529 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [2]:
import requests
from bs4 import BeautifulSoup
import os
from PyPDF2 import PdfReader
from urllib.parse import urlparse

def extract_text(web_url, company):
    try:
        if web_url.lower().endswith('.pdf'):
            pdf_filename = os.path.basename(urlparse(web_url).path)
            
            if not os.path.exists('documents'):
                os.makedirs('documents')
            
            pdf_filepath = os.path.join('documents', pdf_filename)
            
            response = requests.get(web_url)
            with open(pdf_filepath, 'wb') as f:
                f.write(response.content)
            
            with open(pdf_filepath, 'rb') as f:
                reader = PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
            
            print(f'Done for {web_url}')
            return text

        else:
            response = requests.get(web_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()
            print(f'Done for {web_url}')
            return text

    except Exception as e:
        print(f'Failed at {web_url} and {company}: {str(e)}')




In [16]:
from googlesearch import search

def get_google_search_links(query, num_results=15):
    search_results = search(query, num_results=num_results)
    
    links = [result for result in search_results]
    
    return links

In [17]:
get_google_search_links1('abc')


NameError: name 'get_google_search_links1' is not defined

In [10]:
company = input('Enter Company Name')

query = f'{company}+UK+Testimonials'

company_testimonials = bing_search_15(query)
company_testimonials

[]

In [99]:
company_testimonials

[]

In [85]:
import concurrent.futures

company_testimonials = [{
    'company':company,
    'web_url': company_testimonials
}]
def process_testimonial(comp):
    company = comp['company']
    web_sites = []
    for url in comp['web_url']:
        text = extract_text(url, company) 

        if text:
            web_sites.append({'company': company, 'web_content': str(text),'url':url})
    return web_sites

all_web_sites = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_testimonial, company_testimonials)

    for result in results:
        all_web_sites.extend(result)



In [86]:
row_list = []

def process_website(part):
    try:
        text = part['web_content']
        company = part['company']
        url = part['url']
        names = extract_person_names(text,company,url)
        
        return [{'Name': name, 'Company': company,'Testimonial Page':url} for name in names]
    except:
        print(names)
        return []

for web_site in all_web_sites:
    result = process_website(web_site)
    row_list.extend(result)       

In [87]:
len(row_list)

0

In [88]:
row_list

[]

In [89]:
import pandas as pd

df = pd.DataFrame(row_list)


In [90]:
df

""


In [91]:
df.to_excel('Updated_list.xlsx',index=False)

In [92]:
import openpyxl
from openpyxl.utils import get_column_letter

def convert_urls_to_hyperlinks(excel_file, sheet_name, url_column):
    workbook = openpyxl.load_workbook(excel_file)
    sheet = workbook[sheet_name]

    col_letter = get_column_letter(url_column)

    for row in range(2, sheet.max_row + 1): 
        cell = sheet[f"{col_letter}{row}"]
        if isinstance(cell.value, str) and cell.value.startswith("http"):
            sheet[f"{col_letter}{row}"].hyperlink = cell.value
            sheet[f"{col_letter}{row}"].value = 'Link'
            sheet[f"{col_letter}{row}"].style = "Hyperlink"

    workbook.save(excel_file)

excel_file = 'Updated_list.xlsx'
sheet_name = "Sheet1"
url_column = 3  

convert_urls_to_hyperlinks(excel_file, sheet_name, url_column)